Домашнее задание

Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:

- оценка 2 и меньше - низкий рейтинг

- оценка 4 и меньше - средний рейтинг

- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [3]:
import pandas as pd

In [2]:
films_ratings = pd.read_csv('ratings.csv')

In [4]:
def class_rating(ratings):
    if ratings <= 2.0:
        return 'низкий рейтинг'
    elif ratings <= 4.0:
        return 'средний рейтинг'
    elif ratings <= 5.0:
        return 'высокий рейтинг'

In [5]:
films_ratings['class'] = films_ratings['rating'].apply(class_rating)
films_ratings.head()

,userId,movieId,rating,timestamp,class
0,1,307,3.5,1256677221,средний рейтинг
1,1,481,3.5,1256677456,средний рейтинг
2,1,1091,1.5,1256677471,низкий рейтинг
3,1,1257,4.5,1256677460,высокий рейтинг
4,1,1449,4.5,1256677264,высокий рейтинг


Задание 2

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. 

Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [5]:
geo_df = pd.read_csv('keywords.csv')
geo_df[geo_df['keyword'] == 'москва'].head()

,keyword,shows
1259,москва,153050


In [6]:
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

In [7]:
def geo_class(row):
    for region, cities_list in geo_data.items():
        for city in cities_list:
            if city in row['keyword']:
                return region
    return 'undefined'

In [8]:
geo_df['region'] = geo_df.apply(geo_class, axis = 1)
geo_df.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [9]:
geo_df[geo_df['keyword'] == 'сахалин'].head()

,keyword,shows,region
21445,сахалин,14494,Дальний Восток


Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). 

Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

2.1. для каждой строки пройдите по всем годам списка years

2.2. если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска

2.3. если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [2]:
films_ratings = pd.read_csv('ratings.csv')

In [3]:
movies = pd.read_csv('movies.csv')
joined = films_ratings.merge(movies, on='movieId', how='left')

In [36]:
joined.head(1)

,userId,movieId,rating,timestamp,title,genres
0,1,307,3.5,1256677221,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama


In [4]:
years = [x for x in range(1950, 2011)]

In [5]:
def production_year(title_film):
    for year in years:
        if str(year) in title_film:
            return year
    return '1900'

In [ ]:
joined['year'] = joined['title'].apply(production_year) 

In [ ]:
joined.groupby('year').mean()[['rating']].sort_values(by = 'rating', ascending = False)

______________________________________________________________
Ниже тестировала на части файла, т.к. весь файл ноутбук не потянул, вроде бы всё корректно работает

In [37]:
movies[~(movies['title'].str.contains('19')) & ~(movies['title'].str.contains('20'))].head(3) #проверка на то, есть ли фильмы без указания года

,movieId,title,genres
10023,32930,Category 6: Day of Destruction,Action|Drama
10613,40697,Babylon 5,Sci-Fi
15719,79607,"Millions Game, The (Das Millionenspiel)",Action|Drama|Sci-Fi|Thriller


In [38]:
joined[joined['movieId'] == 40697].head(1) #выводим фильм без указания года, чтобы узнать индекс

,userId,movieId,rating,timestamp,title,genres
374582,3846,40697,4.0,1434248236,Babylon 5,Sci-Fi


In [28]:
# в новый файл сохраняем часть датафрейма, куда попадает индекс выбранного фильма
joined_small = joined.loc[374500:374600]
joined_small.to_csv('small.csv')

In [39]:
small_df = pd.read_csv('small.csv')
small_df.head(3)

,Unnamed: 0,userId,movieId,rating,timestamp,title,genres
0,374500,3846,8371,2.5,1436499057,"Chronicles of Riddick, The (2004)",Action|Sci-Fi|Thriller
1,374501,3846,8373,1.0,1436499426,"Stepford Wives, The (2004)",Comedy|Fantasy|Thriller
2,374502,3846,8464,1.5,1436498669,Super Size Me (2004),Comedy|Documentary|Drama


In [42]:
small_df['year'] = small_df['title'].apply(production_year) #добавляем столбец с годом

In [43]:
small_df[small_df['year'] == '1900'].head() #проверяем корректно ли проставились года

,Unnamed: 0,userId,movieId,rating,timestamp,title,genres,year
3,374503,3846,8507,4.0,1434247816,Freaks (1932),Crime|Drama|Horror,1900
82,374582,3846,40697,4.0,1434248236,Babylon 5,Sci-Fi,1900


In [34]:
small_df.groupby('year').mean()[['rating']].sort_values(by = 'rating', ascending = False)

,rating
year,
1965,4.500000
1983,4.500000
1989,4.500000
2003,4.100000
1964,4.000000
1967,4.000000
1985,4.000000
1992,4.000000
1900,4.000000


### Упражнение
Какая кампания имеет наименьшую сумму по столбцу effect? 

In [28]:
sum_effect = stats.groupby('campaign').sum()['effect'].reset_index()
sum_effect.min()

campaign    034ada41b6
effect             2.8
dtype: object